Documentación oficial **Peewee**: https://docs.peewee-orm.com/en/latest/index.html

Mapeo de Objetos en una BBDD
- Model Class => Tabla de la Base de Datos
- Field instance ==> Columna de la tabla
- Model instance => Fila de la tabla.

Para empezar un proyecto de Peewee, lo primero es definir una clase por cada Tabla de nuestra BBDD

In [3]:
from peewee import *

db = SqliteDatabase('people.db')

class Person(Model): # Nombres en SINGULAR ==> En lugar de Personas
    name = CharField()
    birthday = DateField()

    class Meta:
        database = db
        table_name = "personas"

**Nota** El nombre de la tabla por defecto va a ser Person, que es el nombre con el que se ha definido la clase, pero podemos cambiar este valor con el atributo **table_name** en la clase privada META. La tabla se generará en minúscula, si quisieramos modificarlo o incluir camel_case utilizaremos el atributo **legacy_table_names=False**
 
https://docs.peewee-orm.com/en/latest/peewee/models.html#table-names





In [ ]:
class UserProfile(Model): 
    class Meta:
        table_name = 'user_profile_tbl'

In [ ]:
class AdminProfile(Model):
    class meta:
        legacy_table_names=False
# El nombre de la tabla será admin_profile

Para los campos utilizaremos las clases definidas por peewee como Field types table utilizando los tipos de datos de Python, ya que peewee se encarga de convertirlos :

https://docs.peewee-orm.com/en/latest/peewee/models.html#fields

In [4]:
class Pet(Model):
    owner = ForeignKeyField(Person, backref='pets')
    name = CharField()
    animal_type = CharField()

    class Meta:
        database = db

Una vez hemos creado las diferentes clases, utilizamos los métodos que nos permitirán ejecutar las sentencias CRUD con peewee.

In [5]:

db.connect() #Conectar la base de datos.
db.create_tables([Person, Pet])


Con los métodos **save()** y **create()** podemos añadir y actualizar datos de las tablas.

In [ ]:
from datetime import date
uncle_bob = Person(name='Bob', birthday=date(1960, 1, 15))
uncle_bob.save() 

# Con el método save() nos devuelve el número de filas modificadas.

In [ ]:
# Con el método create() nos devuelve una intancia

grandma = Person.create(name='Grandma', birthday=date(1935, 3, 1))
herb = Person.create(name='Herb', birthday=date(1950, 5, 5))

In [ ]:
#Actualizar un dato, hay que hacer un commit ==> para ello utilizamos el método save()

grandma.name = 'Grandma L.'
grandma.save()

In [ ]:
bob_kitty = Pet.create(owner=uncle_bob, name='Kitty', animal_type='cat')
herb_fido = Pet.create(owner=herb, name='Fido', animal_type='dog')
herb_mittens = Pet.create(owner=herb, name='Mittens', animal_type='cat')
herb_mittens_jr = Pet.create(owner=herb, name='Mittens Jr', animal_type='cat')

#eliminar:
herb_mittens.delete_instance()

En las dos clases que hemos definido hemos utilizado la clase privada META para indicar la base de datos a la que está conectado, esto podemos mejorarlo utilizando una clase Base y la herencia

In [ ]:
class BaseModel(Model):
    class Meta:
        database = db

# Hay que tener en cuenta que ahora todas las clases que se definan deben crearse a partir de BaseModel

class Person(BaseModel):
    dni = IntegerField(primary_key=True)
    name = CharField()
    birthday = DataField()

class Pet(BaseModel):
    owner = ForeignKeyField(Person, backref='pets')
    #owner = ForeignKeyField(Person, to_field="dni")
    name = CharField()
    animal_type = CharField()